In [7]:
from datetime import datetime
import json
import os
from integer_linear_problem import Ilp
from config import Config
import pandas as pd
import numpy as np
import math
import scipy.stats as stats
import matplotlib.pyplot as plt
import ilp_solver
from intuitive_algo_1 import intuitive_algo_1
from intuitive_algo_2 import intuitive_algo_2
from solution import Flags
import utils
from tqdm import tqdm


ILP_SMALL = Ilp.from_excel(Config().DATA_SMALL_PATH)
ILP_LARGE = Ilp.from_excel(Config().DATA_LARGE_PATH)

def gen_ilp(n, w_mu, w_sigma, c_mu, c_sigma, f_mu, f_sigma, collection, transfer, distribution):
    w_arr = np.random.normal(loc=w_mu, scale=w_sigma, size=n**2).astype(int)

    # ensure all values are >= 0
    while any(w_arr < 0):
        w_arr[w_arr < 0] = np.random.normal(loc=w_mu, scale=w_sigma, size=sum(w_arr < 0)).astype(int)

    w_iter = iter(w_arr)
    w_dct = dict()
    for i in range(1, n + 1):
        w_dct[i] = dict()
        for j in range(1, n + 1):
            w_dct[i][j] = next(w_iter)

    c_arr = np.random.normal(loc=c_mu, scale=c_sigma, size=n**2 - n).astype(int) # FIXME: the mean and standard deviation don't seem to converge to mu and sigma, https://stackoverflow.com/questions/27831923/python-random-number-generator-with-mean-and-standard-deviation

    # ensure all values are > 0
    # FIXME: this throws the mean and std even more off...
    # FIXME: make costs symmetrical
    while any(c_arr <= 0):
        c_arr[c_arr <= 0] = np.random.normal(loc=c_mu, scale=c_sigma, size=sum(c_arr <= 0)).astype(int)

    c_iter = iter(c_arr)
    c_dct = dict()
    for i in range(1, n + 1):
        c_dct[i] = dict()
        for j in range(1, n + 1):
            if i == j:
                c_dct[i][j] = 0
            else:
                c_dct[i][j] = next(c_iter)

    f_arr = np.random.normal(loc=f_mu, scale=f_sigma, size=n).astype(int)

    # ensure all values are > 0
    while any(f_arr <= 0):
        f_arr[f_arr <= 0] = np.random.normal(loc=f_mu, scale=f_sigma, size=sum(f_arr <= 0)).astype(int)

    f_iter = iter(f_arr)
    f_dct = {i: next(f_iter) for i in range(1, n + 1)}

    N = set(range(1, n + 1))
    ilp = Ilp(N, collection, transfer, distribution, w_dct, c_dct, f_dct)

    return ilp

def get_params_nd():


    # merge small and large data set (also make them of the same size) and calculate mean and standard deviation seperately for flow, costs and fixed costs
    w_small_ser = ILP_SMALL.w_df.stack()
    w_large_ser = ILP_LARGE.w_df.stack()
    
    w_small_ser = pd.concat([w_small_ser] * math.ceil(len(w_large_ser) / len(w_small_ser)))
    w_small_ser = w_small_ser[:len(w_large_ser)]

    w_ser = pd.concat([w_small_ser, w_large_ser])
    w_mu = w_ser.mean()
    w_sigma = w_ser.std()

    # exclude zeros for mean and std
    c_small_ser = ILP_SMALL.c_df.stack()
    c_small_ser = c_small_ser[c_small_ser != 0]
    c_large_ser = ILP_LARGE.c_df.stack()
    c_large_ser = c_large_ser[c_large_ser != 0]

    c_small_ser = pd.concat([c_small_ser] * math.ceil(len(c_large_ser) / len(c_small_ser)))
    c_small_ser = c_small_ser[:len(c_large_ser)]

    c_ser = pd.concat([c_small_ser, c_large_ser])
    c_mu = c_ser.mean()
    c_sigma = c_ser.std()

    f_small_ser = ILP_SMALL.f_ser
    f_large_ser = ILP_LARGE.f_ser

    f_small_ser = pd.concat([f_small_ser] * math.ceil(len(f_large_ser) / len(f_small_ser)))
    f_small_ser = f_small_ser[:len(f_large_ser)]

    f_ser: pd.Series = pd.concat([f_small_ser, f_large_ser])
    f_mu = f_ser.mean()
    f_sigma = f_ser.std()

    return w_mu, w_sigma, c_mu, c_sigma, f_mu, f_sigma

# RUN DIT

In [13]:
ILP = ILP_SMALL
SAMPLES = 50

w_mu, w_sigma, c_mu, c_sigma, f_mu, f_sigma = get_params_nd()

error_arr = []
for _ in tqdm(range(SAMPLES)):
    ilp = gen_ilp(len(ILP.N), w_mu, w_sigma, c_mu, c_sigma, f_mu, f_sigma, ILP.collection, ILP.transfer, ILP.distribution)
    sol_gurobi = ilp_solver.gurobi_v2(ilp, Flags.NONE)
    sol_heuristic_2 = intuitive_algo_2(ilp, Flags.NONE)
    error = (sol_heuristic_2.z - sol_gurobi.z) / sol_gurobi.z * 100
    error_arr.append(error)

print(error_arr)

  0%|          | 0/50 [00:00<?, ?it/s]c:\Users\Thom van den Hil\Desktop\Modelling-B\venv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
calculating (sub)optimal solution: 100it [00:01, 64.18it/s]                                         
calculating (sub)optimal solution: 100it [00:00, 110.99it/s]
calculating (sub)optimal solution: 100it [00:01, 73.64it/s]                                         
calculating (sub)optimal solution: 100it [00:01, 99.70it/s] 
calculating (sub)optimal solution: 100it [00:01, 72.94it/s]                                         
calculating (sub)optimal solution: 100it [00:01, 66.62it/s]                                         
calculating (sub)optimal solution: 100it [00:01, 63.98it/s] 
calculating (sub)optimal solution: 100it [00:01, 73.75it/s]                                         
calculating (sub)optimal solution: 100it [00

[1.8489561036280637, 1.749830543823974, 0.9440167320459374, 0.0, 8.065238158969555, 1.7806188042304816, 2.1594749070833026, 0.5285159749178859, 6.85539217885114, 12.38359923615754, 3.1102725295924323, 2.4820698032681596, 0.0, 9.79655071344496, 1.7735925870202052, 11.536821447731013, 3.9903995231977962, 8.847145846513213, 0.0, 0.0, 0.8315155083887603, 3.062915047017628, 3.3790170132325144, 7.432415830003883, 6.535471678353303, 1.1281215253184342, 1.8841890881450176, 2.6294028843017214, 1.5510124012759496, 5.790279767706121, 2.577206163020976, 1.9007743334930765, 4.2070937683479555, 4.21729836221026, 8.360734963848799, 16.00062813115784, 0.0, 7.362597041442973, 0.0, 0.8732933958598754, 1.6168658698539178, 0.24798542058424014, 0.9754622429583548, 2.0377801039001318, 0.751186926532127, 0.0, 4.9514191950034006, 4.76699416116584, 8.745787875754846, 8.12079113420648]


In [14]:
error_ser = pd.Series(error_arr)
print(error_ser.mean())
print(error_ser.std())

3.795814698471202
3.7647275954223565


In [8]:
ILP = ILP_SMALL
SAMPLES = 10

w_mu, w_sigma, c_mu, c_sigma, f_mu, f_sigma = get_params_nd()

error_arr = []
for _ in tqdm(range(SAMPLES)):
    ilp = gen_ilp(len(ILP.N), w_mu, w_sigma, c_mu, c_sigma, f_mu, f_sigma, ILP.collection, ILP.transfer, ILP.distribution)
    sol_heuristic_1 = intuitive_algo_1(ilp, Flags.NONE)
    sol_heuristic_2 = intuitive_algo_2(ilp, Flags.NONE)
    error = (sol_heuristic_2.z - sol_heuristic_1.z) / sol_heuristic_1.z * 100
    error_arr.append(error)

print(error_arr)

100%|███████████████████████████████████████████████████████████| 1023/1023 [00:31<00:00, 32.92it/s]
calculating (sub)optimal solution: 100it [00:01, 70.42it/s]                                         
100%|███████████████████████████████████████████████████████████| 1023/1023 [00:32<00:00, 31.76it/s]
calculating (sub)optimal solution: 100it [00:01, 77.43it/s] 
100%|███████████████████████████████████████████████████████████| 1023/1023 [00:30<00:00, 33.30it/s]
calculating (sub)optimal solution: 100it [00:01, 85.47it/s] 
100%|███████████████████████████████████████████████████████████| 1023/1023 [00:31<00:00, 32.91it/s]
calculating (sub)optimal solution: 100it [00:01, 60.75it/s]                                         
100%|███████████████████████████████████████████████████████████| 1023/1023 [00:30<00:00, 33.93it/s]
calculating (sub)optimal solution: 100it [00:01, 69.49it/s] 
100%|███████████████████████████████████████████████████████████| 1023/1023 [00:30<00:00, 33.66it/s]
calculati

[0.0, 0.38494748975954607, 3.316403365089058, 1.2322573694018684, 0.0, 3.372916746730205, 25.09423642625907, 0.0, 0.0, 16.119624503728993]


In [9]:
error_ser = pd.Series(error_arr)
print(error_ser.mean())
print(error_ser.std())

4.952038590096874
8.617668384292985
